## Reference Data

In [8]:
reference_data = [
  {
    "question": "What is the company's policy on remote work?", 
    "ground_truth": "Remote work is allowed up to 3 days per week. ", #Expected llm generated answer
    "context": "Remote work is allowed up to 3 days per week." #Expected retrieved context
  }
]
question = reference_data[0]['question']
ground_truth = reference_data[0]['ground_truth']
context = reference_data[0]['context']
print (f"question: {question}")
print (f"ground_truth: {ground_truth}")
print (f"context: {context}")

question: What is the company's policy on remote work?
ground_truth: Remote work is allowed up to 3 days per week. 
context: Remote work is allowed up to 3 days per week.


In [9]:
# Retrieve context from Milvus DB

from milvus_chatbot_with_rag import retrieve_similiar_contexts

def perform_retrieval(question):

    retrieved_context = retrieve_similiar_contexts(question, "employee_policies", 1)[0]['content']
    print (f"perform_retrieval.retrieved_context: {retrieved_context}")
    return retrieved_context

# Validate retrieved context against reference data    
perform_retrieval("What is the company's policy on remote work?")


perform_retrieval.retrieved_context: Remote work is allowed up to 3 days per week.


'Remote work is allowed up to 3 days per week.'

In [10]:
%pip install ragas datasets 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import context_precision, context_recall

from dotenv import load_dotenv
from openai import OpenAI
import os

# --- Load API Key ---
load_dotenv(override=True, dotenv_path="../.env")
my_api_key = os.getenv("OPENAI_API_KEY")


client = OpenAI(api_key=my_api_key)

# question = reference_data[0]['question']
# ground_truth = reference_data[0]['ground_truth']
# context = reference_data[0]['context']
# Question User asked
question = reference_data[0]['question']

# Reference context (should be a string)
reference_context = reference_data[0]['context']

# Retrieved context (a string from perform_retrieval)
retrieved_context = [perform_retrieval(question)]

# Build dataset properly
dataset_dict = {
    "question": [question],
    "contexts": [retrieved_context],      # list of strings INSIDE another list
    "ground_truth": [reference_context],   # single string
    "answer": [""]
}

print(f"dataset_dict: {dataset_dict}")

ragas_dataset = Dataset.from_dict(dataset_dict)

# Evaluate retrieval
results = evaluate(
    dataset=ragas_dataset,
    metrics=[context_precision, context_recall]
)


perform_retrieval.retrieved_context: Remote work is allowed up to 3 days per week.
dataset_dict: {'question': ["What is the company's policy on remote work?"], 'contexts': [['Remote work is allowed up to 3 days per week.']], 'ground_truth': ['Remote work is allowed up to 3 days per week.'], 'answer': ['']}


Evaluating: 100%|██████████| 2/2 [00:02<00:00,  1.28s/it]


In [12]:
print("Retrieval Evaluation Results:")
results.to_pandas()


Retrieval Evaluation Results:


,user_input,retrieved_contexts,response,reference,context_precision,context_recall
0,What is the company's policy on remote work?,[Remote work is allowed up to 3 days per week.],,Remote work is allowed up to 3 days per week.,1.0,1.0


## 